In [ ]:
# Unzipping downloaded files
zipfile.ZipFile(input_path+'zip//test.zip')

import zipfile
for file in os.listdir(input_path):
    if '.7z' in file:
        print(file)
        print(input_path+file)

        with zipfile.ZipFile(input_path + file) as zipArch:
            print(file)
            print(input_path+file)
        #zfile = zipfile.ZipFile(input_path + file) as zipArch
            zipArch.extractall(input_path)
        

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os


notebooks_path = '../nbs/'
input_path = '../input/'
output_path = '../output/'
print(os.listdir(input_path))


# Any results you write to the current directory are saved as output.

['.gitignore', 'sample_submission.csv', 'test.csv', 'train.csv', 'zip']
